In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error

In [ ]:
# Importar datos

df = pd.read_csv("Data_Greener_all.csv")

In [ ]:
print(df.head())

                  Time  Ventilation  Sokets plug  Lighting  Other electricity  \
0  2017-01-03 13:00:00         27.4         41.8      53.7               89.6   
1  2017-01-03 14:00:00         21.6         37.5      50.9               64.6   
2  2017-01-03 15:00:00         18.5         37.4      60.1               64.2   
3  2017-01-03 16:00:00         29.7         38.0      52.6               43.2   
4  2017-01-03 17:00:00         16.7         38.1      56.7               42.4   

   Cooling  Heating  
0     18.6    498.0  
1     61.9    500.0  
2     16.5    480.0  
3     19.8    390.0  
4     15.2    373.0  


In [ ]:
# Normalizar variables

# 1. Convertir Time a datetime
df["Time"] = pd.to_datetime(df["Time"], errors="coerce")

# 2. Convertir las demás columnas a numérico
cols_numericas = ["Ventilation", "Sokets plug", "Lighting", "Other electricity", "Cooling", "Heating"]
df[cols_numericas] = df[cols_numericas].apply(pd.to_numeric, errors="coerce")

# Verificar cambios
print(df.dtypes)

Time                 datetime64[ns]
Ventilation                 float64
Sokets plug                 float64
Lighting                    float64
Other electricity           float64
Cooling                     float64
Heating                     float64
dtype: object


In [ ]:
df["Time"] = pd.to_datetime(df["Time"], errors="coerce")

# Crear nuevas columnas
df['hora'] = df["Time"].dt.hour
df["Dia"] = df["Time"].dt.day
df["Mes"] = df["Time"].dt.month
df["Anno"] = df["Time"].dt.year
df["DiaSemanaNum"] = df["Time"].dt.weekday   # 0 = Lunes, 6 = Domingo
df["DiaSemana"] = df["Time"].dt.day_name()   # Nombre del día


df["Total"] = (
    df["Ventilation"]
    + df["Sokets plug"]
    + df["Lighting"]
    + df["Other electricity"]
    + df["Cooling"]
    + df["Heating"]
)

print(df.head())

                 Time  Ventilation  Sokets plug  Lighting  Other electricity  \
0 2017-01-03 13:00:00         27.4         41.8      53.7               89.6   
1 2017-01-03 14:00:00         21.6         37.5      50.9               64.6   
2 2017-01-03 15:00:00         18.5         37.4      60.1               64.2   
3 2017-01-03 16:00:00         29.7         38.0      52.6               43.2   
4 2017-01-03 17:00:00         16.7         38.1      56.7               42.4   

   Cooling  Heating  hora  Dia  Mes  Anno  DiaSemanaNum DiaSemana  Total  
0     18.6    498.0    13    3    1  2017             1   Tuesday  729.1  
1     61.9    500.0    14    3    1  2017             1   Tuesday  736.5  
2     16.5    480.0    15    3    1  2017             1   Tuesday  676.7  
3     19.8    390.0    16    3    1  2017             1   Tuesday  573.3  
4     15.2    373.0    17    3    1  2017             1   Tuesday  542.1  


In [ ]:
# Aseguramos que "Time" sea índice de tiempo
df["Time"] = pd.to_datetime(df["Time"])
df = df.set_index("Time")

df_ori=df.copy()
# Variables a tratar
variables = ["Sokets plug", "Lighting", "Other electricity", "Cooling", "Heating"]

for col in variables:
    # Calcular límites con rango intercuartílico (IQR)
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    # Reemplazar valores atípicos por NaN
    df.loc[(df[col] < lower) | (df[col] > upper), col] = None

# Interpolación temporal
df[variables] = df[variables].interpolate(method='time')

# (Opcional) Rellenar NaN restantes al inicio o final
df[variables] = df[variables].fillna(method='bfill').fillna(method='ffill')


/tmp/ipython-input-655676188.py:24: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[variables] = df[variables].fillna(method='bfill').fillna(method='ffill')


In [ ]:
##### Máquinas de Soporte Vectorial (SVR) Heating / Calefacción #####

# Importar librerías
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Variables predictoras y objetivo
X = df[["Dia", "Mes", "Anno", "DiaSemanaNum", "Total" ,"hora"]]
y = df["Heating"]

# Escalado de las variables (muy importante para SVM)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

# Dividir datos en entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Crear modelo SVM con kernel RBF (común en regresión)
model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_scaled = model.predict(X_test)

# Invertir la escala de la predicción para comparar con valores reales
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_test_real = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Evaluar el modelo
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred))
r2 = r2_score(y_test_real, y_pred)
mae = mean_absolute_error(y_test_real, y_pred)

print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")


RMSE: 21.689
R²: 0.891
MAE: 13.050


In [ ]:
##### Máquinas de Soporte Vectorial (SVR) Lighting/Iluminación #####

# Variables predictoras y objetivo
X = df[["Dia", "Mes", "Anno", "DiaSemanaNum", "Total" ,"hora"]]
y = df["Lighting"]

# Escalado de las variables (muy importante para SVM)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

# Dividir datos en entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Crear modelo SVM con kernel RBF (común en regresión)
model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_scaled = model.predict(X_test)

# Invertir la escala de la predicción para comparar con valores reales
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_test_real = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Evaluar el modelo
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred))
r2 = r2_score(y_test_real, y_pred)
mae = mean_absolute_error(y_test_real, y_pred)

print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")

RMSE: 5.123
R²: 0.872
MAE: 3.375


In [ ]:
##### Máquinas de Soporte Vectorial (SVR) Cooling/Enfriamiento #####

# Variables predictoras y objetivo
X = df[["Dia", "Mes", "Anno", "DiaSemanaNum", "Total" ,"hora"]]
y = df["Cooling"]

# Escalado de las variables (muy importante para SVM)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

# Dividir datos en entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Crear modelo SVM con kernel RBF (común en regresión)
model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_scaled = model.predict(X_test)

# Invertir la escala de la predicción para comparar con valores reales
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_test_real = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Evaluar el modelo
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred))
r2 = r2_score(y_test_real, y_pred)
mae = mean_absolute_error(y_test_real, y_pred)

print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")

RMSE: 13.557
R²: 0.764
MAE: 9.113


In [ ]:
##### Máquinas de Soporte Vectorial enchufe de toma corriente(SVR) #####

# Variables predictoras y objetivo
X = df[["Dia", "Mes", "Anno", "DiaSemanaNum", "Total" ,"hora"]]
y = df["Sokets plug"]

# Escalado de las variables (muy importante para SVM)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

# Dividir datos en entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Crear modelo SVM con kernel RBF (común en regresión)
model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_scaled = model.predict(X_test)

# Invertir la escala de la predicción para comparar con valores reales
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_test_real = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Evaluar el modelo
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred))
r2 = r2_score(y_test_real, y_pred)
mae = mean_absolute_error(y_test_real, y_pred)

print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")

RMSE: 3.427
R²: 0.757
MAE: 2.298


In [11]:
##### Máquinas de Soporte Vectorial (SVR) Other electricity/ Otros electrónicos #####

# Variables predictoras y objetivo
X = df[["Dia", "Mes", "Anno", "DiaSemanaNum", "Total" ,"hora"]]
y = df["Other electricity"]

# Escalado de las variables (muy importante para SVM)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

# Dividir datos en entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Crear modelo SVM con kernel RBF (común en regresión)
model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_scaled = model.predict(X_test)

# Invertir la escala de la predicción para comparar con valores reales
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_test_real = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Evaluar el modelo
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred))
r2 = r2_score(y_test_real, y_pred)
mae = mean_absolute_error(y_test_real, y_pred)

print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")

RMSE: 28.052
R²: 0.611
MAE: 18.904


In [12]:
##### Máquinas de Soporte Vectorial (SVR) Ventilation #####



# Variables predictoras y objetivo
X = df[["Dia", "Mes", "Anno", "DiaSemanaNum", "Total" ,"hora"]]
y = df["Ventilation"]

# Escalado de las variables (muy importante para SVM)
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1)).ravel()

# Dividir datos en entrenamiento y prueba (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Crear modelo SVM con kernel RBF (común en regresión)
model = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=0.1)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred_scaled = model.predict(X_test)

# Invertir la escala de la predicción para comparar con valores reales
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1))
y_test_real = scaler_y.inverse_transform(y_test.reshape(-1, 1))

# Evaluar el modelo
rmse = np.sqrt(mean_squared_error(y_test_real, y_pred))
r2 = r2_score(y_test_real, y_pred)
mae = mean_absolute_error(y_test_real, y_pred)

print(f"RMSE: {rmse:.3f}")
print(f"R²: {r2:.3f}")
print(f"MAE: {mae:.3f}")

RMSE: 23.442
R²: 0.558
MAE: 11.319


La categoría de consumo energético que mejor se puede desagregar (predecir) con es (tiempo, total, y Heating) es Cooling (Refrigeración/Aire Acondicionado), seguida de Lighting (Iluminación).


*   Lighting ($\mathbf{R^2 = 0.87}$): El modelo explica aproximadamente el $87\%$ de su variabilidad. Esto es lógico, ya que la iluminación depende casi exclusivamente de la hora del día (y mes/estación), lo cual está bien capturado por las características temporales (hour_sin, month_cos, etc.).

*  Enfriamient ($\mathbf{R^2 = 0.764}$): El modelo SVR explica casi el $76\%$ de la variabilidad en el consumo de refrigeración. Esto indica que el patrón de uso de la refrigeración está fuertemente relacionado con la hora del día, el mes y el consumo general (Total y Heating). Este es un resultado muy bueno, lo que significa que el SVR puede hacer predicciones confiables para esta categoría.

* Se propone añadir datos externo del API externas del clima que son corelacionados con las variables calefacción y enfiramiento https://content.meteoblue.com/es/soluciones-para-empresas/api-meteorologica/free-weather-api


